In [21]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from tqdm import tqdm

load_dotenv()  
URI = os.environ["NEO4J_INSTANCE01_URI"]
USER = os.environ["NEO4J_INSTANCE01_USER"]
PWD = os.environ["NEO4J_INSTANCE01_KEY"]

graph = Neo4jGraph(url=URI, username=USER, password=os.environ["NEO4J_INSTANCE01_KEY"])



In [9]:
with open('cypher_queries.txt','r') as f:
    queries = f.readlines()
print(queries)

['CREATE (:article {name: "Article"});\n', 'CREATE (:section {name: "Abstract"});\n', 'MATCH (a:article{name:"Article"}), (b:section {name:"Abstract"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:paragraph {name: "Abstract P1"});\n', 'MATCH (a:section{name:"Abstract"}), (b:paragraph {name:"Abstract P1"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:section {name: "1. Introduction"});\n', 'MATCH (a:article{name:"Article"}), (b:section {name:"1. Introduction"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:paragraph {name: "1. P1"});\n', 'MATCH (a:section{name:"1. Introduction"}), (b:paragraph {name:"1. P1"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:section {name: "2. Science of the Moon"});\n', 'MATCH (a:article{name:"Article"}), (b:section {name:"2. Science of the Moon"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:paragraph {name: "2. P1"});\n', 'MATCH (a:section{name:"2. Science of the Moon"}), (b:paragraph {name:"2. P1"})  CREATE (b)-[:IS_PART_OF]->(a);\n', 'CREATE (:parag

In [10]:
for query in tqdm(queries):
    graph.query(query)
    

  0%|          | 0/667 [00:00<?, ?it/s]

100%|██████████| 667/667 [00:39<00:00, 17.02it/s]


In [22]:
chain = GraphCypherQAChain.from_llm(
     graph=graph,
     cypher_llm=ChatOpenAI(temperature=0, model="gpt-4", openai_api_key= os.environ["OPENAI_API_KEY"]),
     qa_llm=ChatOpenAI(temperature=0, model="gpt-4", openai_api_key= os.environ["OPENAI_API_KEY"]),
     verbose=True,
     return_intermediate_steps=True,
     top_k=100
)

In [16]:
print(graph.schema)


        Node properties are the following:
        [{'labels': 'article', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'section', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'paragraph', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'subsection', 'properties': [{'property': 'name', 'type': 'STRING'}]}, {'labels': 'concept', 'properties': [{'property': 'name', 'type': 'STRING'}]}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:section)-[:IS_PART_OF]->(:article)', '(:paragraph)-[:IS_PART_OF]->(:section)', '(:paragraph)-[:IS_PART_OF]->(:subsection)', '(:paragraph)-[:TALKS_ABOUT]->(:concept)', '(:subsection)-[:IS_PART_OF]->(:section)', '(:concept)-[:PROVIDES_INFORMATION_ABOUT]->(:concept)', '(:concept)-[:OFFERS_OPPORTUNITIES_FOR]->(:concept)', '(:concept)-[:CAN_BE_PLACED_AND_OPERATED_ON]->(:concept)', '(:concept)-[:REQUIRED_TO_ADDRESS]->(:concept)', 

In [23]:
chain.run(" What is the abstract about?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:paragraph)-[:TALKS_ABOUT]->(c:concept) RETURN p.name, c.name
Full Context:
[{'p.name': 'Abstract P1', 'c.name': 'lunar geological record'}, {'p.name': 'Abstract P1', 'c.name': 'Solar System'}, {'p.name': 'Abstract P1', 'c.name': 'origin and evolution of the Earth-Moon system'}, {'p.name': 'Abstract P1', 'c.name': 'geological evolution of rocky planets'}, {'p.name': 'Abstract P1', 'c.name': 'near-Earth cosmic environment'}, {'p.name': 'Abstract P1', 'c.name': 'lunar surface'}, {'p.name': 'Abstract P1', 'c.name': 'research in astronomy'}, {'p.name': 'Abstract P1', 'c.name': 'astrobiology'}, {'p.name': 'Abstract P1', 'c.name': 'fundamental physics'}, {'p.name': 'Abstract P1', 'c.name': 'life sciences'}, {'p.name': 'Abstract P1', 'c.name': 'human physiology and medicine'}, {'p.name': 'Abstract P1', 'c.name': 'lunar science objectives'}, {'p.name': 'Abstract P1', 'c.name': 'lunar surface exploration'}, {'p.name': 'Abstr

'The abstract is about lunar science objectives and exploration. It discusses the geological evolution of rocky planets, the origin and evolution of the Earth-Moon system, and the lunar geological record. It also covers the use of scientific instruments in lunar surface exploration, both through human operations and robotic exploration. The abstract further delves into the broader context of space exploration, including astrobiology, fundamental physics, life sciences, and space medicine. It also mentions the involvement of various countries and their respective space agencies, such as the European Space Agency, Japan, China, India, and the United States, in lunar exploration programs.'

'The abstract is about lunar science objectives and exploration. It discusses the geological evolution of rocky planets, the origin and evolution of the Earth-Moon system, and the lunar geological record. It also covers the use of scientific instruments in lunar surface exploration, both through human operations and robotic exploration. The abstract further delves into the broader context of space exploration, including astrobiology, fundamental physics, life sciences, and space medicine. It also mentions the involvement of various countries and their respective space agencies, such as the European Space Agency, Japan, China, India, and the United States, in lunar exploration programs.'
